In [4]:
# create pickles of ads with features extracted

import sys
sys.path.append("/Users/david/Documents/mastersCode/ubm")
print(sys.path)

['/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug', '/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev', '/Users/david/Documents/mastersCode/ubm/data/pitch_tracking_database', '/Users/david/.conda/envs/ubm/lib/python310.zip', '/Users/david/.conda/envs/ubm/lib/python3.10', '/Users/david/.conda/envs/ubm/lib/python3.10/lib-dynload', '', '/Users/david/.conda/envs/ubm/lib/python3.10/site-packages', '/Users/david/Documents/mastersCode/ubm', '/Users/david/Documents/mastersCode/ubm']


In [5]:
# test for gmm-ubm on whale calls
from importlib import reload
import audio_datastore.audio_datastore as myads
reload(myads)
from collections import Counter
import numpy as np
import config
import librosa
import utils as my_utils
import feature_extraction.fe_spafe.fe_spafe as fe_spafe
import feature_extraction.fe_base as fe_base
import os.path
reload(fe_base)
reload(fe_spafe)


<module 'feature_extraction.fe_spafe.fe_spafe' from '/Users/david/Documents/mastersCode/ubm/feature_extraction/fe_spafe/fe_spafe.py'>

In [6]:
# create ads

# female ads
datasetFolder = r"/Users/david/Documents/data/speech/ivectors/SPEECH DATA/FEMALE/MIC"
female_ads = myads.AudioDatastore()
female_ads.populate(datasetFolder, include_sub_folders=True, label_source=True)
Counter(female_ads.labels).values()
# male ads
datasetFolder = r"/Users/david/Documents/data/speech/ivectors/SPEECH DATA/MALE/MIC"
male_ads = myads.AudioDatastore()
male_ads.populate(datasetFolder, include_sub_folders=True, label_source=True)
Counter(male_ads.labels).values()
# combine ads
ads = myads.AudioDatastore()
ads.set([datasetFolder], files=female_ads.files + male_ads.files, labels=female_ads.labels + male_ads.labels)
# split ads
all_labels = ["M01", "M02", "M03", "M04", "M06", "M07", "M08", "M09", "F01", "F02", "F03", "F04", "F06", "F07",
                     "F08", "F09", "M05", "M10", "F05", "F10"]

ads_all = myads.subset(ads, all_labels)
ads_all, _ = myads.split(ads_all, 30)

In [7]:
from collections import Counter
print(Counter(ads_all.labels).keys()) # equals to list(set(words))
print(Counter(ads_all.labels).values()) # counts the elements' frequency

dict_keys(['F01', 'F02', 'F03', 'F04', 'F05', 'F06', 'F07', 'F08', 'F09', 'F10', 'M01', 'M02', 'M03', 'M04', 'M05', 'M06', 'M07', 'M08', 'M09', 'M10'])
dict_values([30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30])


In [8]:
# define torch transforms for normal file conversion

import my_torch.tuts2.torch_transforms as torch_t
import my_torch.tuts2.combo_torch_transforms as torch_c
reload(torch_t)
reload(torch_c)

classic_process = torch_c.file_to_numpy()

Using cache found in /Users/david/.cache/torch/hub/snakers4_silero-vad_master


In [9]:
import feature_extraction.fe_base
# fe methods
from feature_extraction.fe_configs import NormFactor
from spafe.features.mfcc import *
reload(fe_base)

def get_mfcc(sign):
    features = mfcc(sig=sign, fs=config.SAMPLING_RATE, num_ceps=config.N_MFCC, nfft=config.N_FFT)
    features = zero_handling(features)
    return features

def get_log_mel(sign):
    features, _ = mel_spectrogram(sig=sign, fs=config.SAMPLING_RATE, nfft=config.N_FFT)
    features = zero_handling(features)
    log_features = np.log(features)
    return log_features

def get_delta_mfccs(sign):
    features = get_mfcc(sign)
    delta_features = librosa.feature.delta(features.T, order=1).T
    features_comb = np.concatenate([features, delta_features], axis=1)
    return features_comb

class FeatureExtractorMfcc(fe_base.FeatureExtractorBase):

    def __init__(self, normalize=False):
        super().__init__(normalize)

    def __str__(self):
        return 'mfcc'

    def extract_feature(self, sig):
        features = mfcc(sig=sig, fs=config.SAMPLING_RATE, num_ceps=config.N_MFCC, nfft=config.N_FFT)
        features = zero_handling(features)
        return features

In [10]:
# quick test - use ads for normalisation and average power if augmenting or adding noise
import matplotlib.pyplot as plt
import spafe.utils.vis as vis
import torch

ads_normalize, _ = myads.split(ads_all, 2) # use one speaker
fe_method = FeatureExtractorMfcc()
fe_method.set_normalisation(ads_normalize, classic_process, verbose=True)

processed files:  10
processed files:  20
average power:  0.016570018892731627


In [11]:
train_process = torch_t.ComposeTransform([
    classic_process,
    fe_method
])

In [17]:
feature_test = train_process(ads_all.files[0])

In [18]:
# ads_train_subset, _ = myads.split(ads_all, 2)
all_features = []
count = 0
for file in ads_all.files:
    if count % 10 == 0:
        print('processed files: ' + str(count) + '/' + str(len(ads_all.files)))
    feature = classic_process(file)
    all_features.append(feature)
    count = count + 1
print('done')
ads_all.data = all_features

processed files: 0/600
processed files: 10/600
processed files: 20/600
processed files: 30/600
processed files: 40/600
processed files: 50/600
processed files: 60/600
processed files: 70/600
processed files: 80/600
processed files: 90/600
processed files: 100/600
processed files: 110/600
processed files: 120/600
processed files: 130/600
processed files: 140/600
processed files: 150/600
processed files: 160/600
processed files: 170/600
processed files: 180/600
processed files: 190/600
processed files: 200/600
processed files: 210/600
processed files: 220/600
processed files: 230/600
processed files: 240/600
processed files: 250/600
processed files: 260/600
processed files: 270/600
processed files: 280/600
processed files: 290/600
processed files: 300/600
processed files: 310/600
processed files: 320/600
processed files: 330/600
processed files: 340/600
processed files: 350/600
processed files: 360/600
processed files: 370/600
processed files: 380/600
processed files: 390/600
processed f

In [19]:
ads_and_info = {
    'ads': ads_all,
    'norm': None,
    'average_power': None,
    'fe_method': None,
    'info': 'pitch tracking database, normalised audio data, sped up 30 examples each'
}

In [21]:
import pickle
pickle.dump(ads_and_info, open('pitch_tracking_db_normalised_audio_30_v2.pickle','wb'))

# adsEnrollSaved = pickle.load(open('adsEnroll.pickle','rb'))